In [7]:
%pip install "pyautogen>=0.2.3"
%pip install chromadb
%pip install sentence_transformers
%pip install tiktoken
%pip install pypdf
%pip install markdownify
%pip install nest-asyncio

^C
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/__main__.py", line 24, in <module>
    sys.exit(_main())
             ^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 77, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", 

In [3]:
import asyncio
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

import nest_asyncio
import autogen

from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.agentchat.user_proxy_agent import UserProxyAgent

In [4]:
# Define an asynchronous function that simulates some asynchronous task (e.g., I/O operation)


async def my_asynchronous_function():
    print("Start asynchronous function")
    await asyncio.sleep(2)  # Simulate some asynchronous task (e.g., I/O operation)
    print("End asynchronous function")
    return "input"


# Define a custom class `CustomisedUserProxyAgent` that extends `UserProxyAgent`


class CustomisedUserProxyAgent(UserProxyAgent):
    # Asynchronous function to get human input
    async def a_get_human_input(self, prompt: str) -> str:
        # Call the asynchronous function to get user input asynchronously
        user_input = await my_asynchronous_function()

        return user_input

    # Asynchronous function to receive a message

    async def a_receive(
        self,
        message: Union[Dict, str],
        sender,
        request_reply: Optional[bool] = None,
        silent: Optional[bool] = False,
    ):
        # Call the superclass method to handle message reception asynchronously
        await super().a_receive(message, sender, request_reply, silent)


class CustomisedAssistantAgent(AssistantAgent):
    # Asynchronous function to get human input
    async def a_get_human_input(self, prompt: str) -> str:
        # Call the asynchronous function to get user input asynchronously
        user_input = await my_asynchronous_function()

        return user_input

    # Asynchronous function to receive a message
    async def a_receive(
        self,
        message: Union[Dict, str],
        sender,
        request_reply: Optional[bool] = None,
        silent: Optional[bool] = False,
    ):
        # Call the superclass method to handle message reception asynchronously
        await super().a_receive(message, sender, request_reply, silent)

In [5]:
config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
)

In [4]:
nest_asyncio.apply()


async def main():
    boss = CustomisedUserProxyAgent(
        name="boss",
        human_input_mode="ALWAYS",
        max_consecutive_auto_reply=1,
        code_execution_config=False,
    )

    assistant = CustomisedAssistantAgent(
        name="assistant",
        system_message="You will provide some agenda, and I will create questions for an interview meeting. Every time when you generate question then you have to ask user for feedback and if user provides the feedback then you have to incorporate that feedback and generate new set of questions and if user don't want to update then terminate the process and exit",
        llm_config={"config_list": config_list, "seed": 23, "temperature": 0.4},
    )

    await boss.a_initiate_chat(
        assistant,
        message="Resume Review, Technical Skills Assessment, Project Discussion, Job Role Expectations, Closing Remarks.",
        n_results=3,
    )


await main() # noqa: F704

boss (to assistant):

Resume Review, Technical Skills Assessment, Project Discussion, Job Role Expectations, Closing Remarks.

--------------------------------------------------------------------------------
assistant (to boss):

Great, let's start with the questions for each agenda item.

### Resume Review
1. Can you walk us through your resume and highlight your most relevant experiences?
2. What accomplishments are you most proud of in your previous roles?

### Technical Skills Assessment
3. Can you describe a challenging technical problem you’ve encountered and how you solved it?
4. Which programming languages and tools are you most proficient in, and how have you applied them in your past projects?

### Project Discussion
5. Can you discuss a recent project you worked on and your role in it?
6. What were the key challenges you faced during this project, and how did you overcome them?

### Job Role Expectations
7. What interests you about this role and our company?
8. How do you se

CancelledError: 